In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
simp_path = '.env'
abs_path = os.path.abspath(simp_path)
CREDENTIAL={'EMAIL':'','PASSWORD':''}
with open(abs_path,"r") as archivo:
    for linea in archivo:        
        if (str(linea.strip('\n')).split('='))[0] == "EMAIL":
            CREDENTIAL["EMAIL"]=(str(linea.strip('\n')).split('='))[1]
        if (str(linea.strip('\n')).split('='))[0] == "PASSWORD":
            CREDENTIAL["PASSWORD"]=(str(linea.strip('\n')).split('='))[1]
            
            
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#abrir formulario

driver.get('https://sivireno.undc.edu.pe/index_login.php')
driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/form/div[1]/a').click()
driver.find_element(by=By.XPATH, value='//*[@id="identifierId"]').send_keys(CREDENTIAL["EMAIL"])
driver.find_element(by=By.XPATH, value='//*[@id="identifierNext"]/div/button/span').click()
for j in range(10):
    try:
        driver.find_element(by=By.XPATH, value='//*[@id="password"]/div[1]/div/div[1]/input').send_keys(CREDENTIAL["PASSWORD"])
        driver.find_element(by=By.XPATH, value='//*[@id="passwordNext"]/div/button/span').click()
        break
    except:
        time.sleep(1)

In [5]:
#SCRIPT CARGA NO LECTIVA DE MANERA MASIVA
import pandas as pd
import datetime
df = pd.read_excel('EXCEL_TRABAJADO/2023-2cnl.xlsx')
df
semanas_xpath= {'1':'//*[@id="exampleFormControlSelect1"]/option[1]',
               '2':'//*[@id="exampleFormControlSelect1"]/option[2]',
               '3':'//*[@id="exampleFormControlSelect1"]/option[3]',
               '4':'//*[@id="exampleFormControlSelect1"]/option[4]',
               '5':'//*[@id="exampleFormControlSelect1"]/option[5]',
               '6':'//*[@id="exampleFormControlSelect1"]/option[6]',
               '7':'//*[@id="exampleFormControlSelect1"]/option[7]',
               '8':'//*[@id="exampleFormControlSelect1"]/option[8]',
               '9':'//*[@id="exampleFormControlSelect1"]/option[9]',
               '10':'//*[@id="exampleFormControlSelect1"]/option[10]',
               '11':'//*[@id="exampleFormControlSelect1"]/option[11]',
               '12':'//*[@id="exampleFormControlSelect1"]/option[12]',
               '13':'//*[@id="exampleFormControlSelect1"]/option[13]',
               '14':'//*[@id="exampleFormControlSelect1"]/option[14]',
               '15':'//*[@id="exampleFormControlSelect1"]/option[15]',
               '16':'//*[@id="exampleFormControlSelect1"]/option[16]',
               '17':'//*[@id="exampleFormControlSelect1"]/option[17]',
               '18':'//*[@id="exampleFormControlSelect1"]/option[18]',
               '19':'//*[@id="exampleFormControlSelect1"]/option[19]',
               '20':'//*[@id="exampleFormControlSelect1"]/option[20]',
               '21':'//*[@id="exampleFormControlSelect1"]/option[21]',
               '22':'//*[@id="exampleFormControlSelect1"]/option[22]',
               '23':'//*[@id="exampleFormControlSelect1"]/option[23]',
               '24':'//*[@id="exampleFormControlSelect1"]/option[24]',
               '25':'//*[@id="exampleFormControlSelect1"]/option[25]',
               '26':'//*[@id="exampleFormControlSelect1"]/option[26]'}

#INICIO DE CLASES
w_iclass={'LUNES':'25/03/2024',
               'MARTES':'26/03/2024',
               'MIERCOLES':'27/03/2024',
               'JUEVES':'28/03/2024',
               'VIERNES':'29/03/2024',
               'SABADO':'30/03/2024',
               'DOMINGO':'31/03/2024'}

urltemp=""
############################################            METODOS
    #ALGORITMO PARA LA FECHA
def convertDateWrite(fecha):
    f = str(fecha).split('-')
    año = f[0]
    mes = f[1]
    d= f[2].split()
    dia = d[0]
    return dia+mes+año
    
    #ALGORTIMO HORA
def convertHourWrite(hour):
    h = str(hour).split(':')
    return h[0]+h[1]

    # 7 + W, CON RESPECTO AL DIA
def sumWeek(dia,sem):
    return convertDateWrite(datetime.datetime.strptime(w_iclass[dia], '%d/%m/%Y')+(datetime.timedelta(days=7)*(sem-1)))

def writeForm(semi,semf,dia,doc,horae,horas):
    for j in range(int(semi),int(semf+1)):
        print("Docente :"+doc+" j:"+str(j)+" semi:"+str(semi)+" semf:"+str(semf)+" fecha i:"+sumWeek(dia,j))
        time.sleep(0.140)
        #ingresar semana
        driver.find_element(by=By.XPATH, value=semanas_xpath[str(int(j))]).click()
        time.sleep(0.150)
        #ingresar fecha
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[2]/input').send_keys(sumWeek(dia,j))

        #ingresar fecha entrada
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[3]/input').send_keys(str(convertHourWrite(horae)))
        #ingresar fecha salida
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[4]/input').send_keys(str(convertHourWrite(horas)))

        #enviar registro
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_nolectiva"]/div[3]/div[2]/table/tbody/tr[2]/td[9]/button').click()
        time.sleep(0.150)
############################################            METODOS/*

for row, datos in df.iterrows():
    if datos['FLAG'] == 'SI':
        url = datos['URL']
        semi = datos['SEM_I']
        semf = datos['SEM_F']
        dia = datos['DIA']
        doc=datos['DOCENTE']
        horae = datos['HORAE']
        horas = datos['HORAS']


        if urltemp==url:
            writeForm(semi,semf,dia,doc,horae,horas)
        else:
            if str(semi)!="nan":   
                urltemp=url
                time.sleep(2)
                driver.get(url)
                time.sleep(1.5)
                writeForm(semi,semf,dia,doc,horae,horas)
            

Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:1 semi:1.0 semf:17.0 fecha i:27032023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:2 semi:1.0 semf:17.0 fecha i:03042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:3 semi:1.0 semf:17.0 fecha i:10042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:4 semi:1.0 semf:17.0 fecha i:17042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:5 semi:1.0 semf:17.0 fecha i:24042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:6 semi:1.0 semf:17.0 fecha i:01052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:7 semi:1.0 semf:17.0 fecha i:08052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:8 semi:1.0 semf:17.0 fecha i:15052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:9 semi:1.0 semf:17.0 fecha i:22052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:10 semi:1.0 semf:17.0 fecha i:29052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:11 semi:1.0 semf:17.0 fecha i:05062023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:12 semi:1.0 semf:17.0 fecha i:12062023
D

Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:15 semi:1.0 semf:17.0 fecha i:05072023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:16 semi:1.0 semf:17.0 fecha i:12072023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:17 semi:1.0 semf:17.0 fecha i:19072023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:1 semi:1.0 semf:17.0 fecha i:01042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:2 semi:1.0 semf:17.0 fecha i:08042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:3 semi:1.0 semf:17.0 fecha i:15042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:4 semi:1.0 semf:17.0 fecha i:22042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:5 semi:1.0 semf:17.0 fecha i:29042023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:6 semi:1.0 semf:17.0 fecha i:06052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:7 semi:1.0 semf:17.0 fecha i:13052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:8 semi:1.0 semf:17.0 fecha i:20052023
Docente :VALDERRAMA ROMERO ANTONIO SALOMON j:9 semi:1.0 semf:17.0 fecha i:27052023
D

Docente :MATOS LIZANA JULIO CESAR j:5 semi:1.0 semf:17.0 fecha i:24042023
Docente :MATOS LIZANA JULIO CESAR j:6 semi:1.0 semf:17.0 fecha i:01052023
Docente :MATOS LIZANA JULIO CESAR j:7 semi:1.0 semf:17.0 fecha i:08052023
Docente :MATOS LIZANA JULIO CESAR j:8 semi:1.0 semf:17.0 fecha i:15052023
Docente :MATOS LIZANA JULIO CESAR j:9 semi:1.0 semf:17.0 fecha i:22052023
Docente :MATOS LIZANA JULIO CESAR j:10 semi:1.0 semf:17.0 fecha i:29052023
Docente :MATOS LIZANA JULIO CESAR j:11 semi:1.0 semf:17.0 fecha i:05062023
Docente :MATOS LIZANA JULIO CESAR j:12 semi:1.0 semf:17.0 fecha i:12062023
Docente :MATOS LIZANA JULIO CESAR j:13 semi:1.0 semf:17.0 fecha i:19062023
Docente :MATOS LIZANA JULIO CESAR j:14 semi:1.0 semf:17.0 fecha i:26062023
Docente :MATOS LIZANA JULIO CESAR j:15 semi:1.0 semf:17.0 fecha i:03072023
Docente :MATOS LIZANA JULIO CESAR j:16 semi:1.0 semf:17.0 fecha i:10072023
Docente :MATOS LIZANA JULIO CESAR j:17 semi:1.0 semf:17.0 fecha i:17072023
Docente :MATOS LIZANA JULIO CE

Docente :MONTERO RAVELO ALEXEI ARMANDO j:11 semi:1.0 semf:17.0 fecha i:08062023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:12 semi:1.0 semf:17.0 fecha i:15062023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:13 semi:1.0 semf:17.0 fecha i:22062023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:14 semi:1.0 semf:17.0 fecha i:29062023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:15 semi:1.0 semf:17.0 fecha i:06072023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:16 semi:1.0 semf:17.0 fecha i:13072023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:17 semi:1.0 semf:17.0 fecha i:20072023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:1 semi:1.0 semf:17.0 fecha i:31032023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:2 semi:1.0 semf:17.0 fecha i:07042023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:3 semi:1.0 semf:17.0 fecha i:14042023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:4 semi:1.0 semf:17.0 fecha i:21042023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:5 semi:1.0 semf:17.0 fecha i:28042023
Docente :MONTERO RAVELO ALEXEI ARMANDO j:6 se

Docente :LEON TTACCA BETSABE j:4 semi:1.0 semf:17.0 fecha i:17042023
Docente :LEON TTACCA BETSABE j:5 semi:1.0 semf:17.0 fecha i:24042023
Docente :LEON TTACCA BETSABE j:6 semi:1.0 semf:17.0 fecha i:01052023
Docente :LEON TTACCA BETSABE j:7 semi:1.0 semf:17.0 fecha i:08052023
Docente :LEON TTACCA BETSABE j:8 semi:1.0 semf:17.0 fecha i:15052023
Docente :LEON TTACCA BETSABE j:9 semi:1.0 semf:17.0 fecha i:22052023
Docente :LEON TTACCA BETSABE j:10 semi:1.0 semf:17.0 fecha i:29052023
Docente :LEON TTACCA BETSABE j:11 semi:1.0 semf:17.0 fecha i:05062023
Docente :LEON TTACCA BETSABE j:12 semi:1.0 semf:17.0 fecha i:12062023
Docente :LEON TTACCA BETSABE j:13 semi:1.0 semf:17.0 fecha i:19062023
Docente :LEON TTACCA BETSABE j:14 semi:1.0 semf:17.0 fecha i:26062023
Docente :LEON TTACCA BETSABE j:15 semi:1.0 semf:17.0 fecha i:03072023
Docente :LEON TTACCA BETSABE j:16 semi:1.0 semf:17.0 fecha i:10072023
Docente :LEON TTACCA BETSABE j:17 semi:1.0 semf:17.0 fecha i:17072023
Docente :LEON TTACCA BETSA

...

Docente :TAIPE CANCHO MARIO HUMBERTO j:11 semi:1.0 semf:17.0 fecha i:05062023
Docente :TAIPE CANCHO MARIO HUMBERTO j:12 semi:1.0 semf:17.0 fecha i:12062023
Docente :TAIPE CANCHO MARIO HUMBERTO j:13 semi:1.0 semf:17.0 fecha i:19062023
Docente :TAIPE CANCHO MARIO HUMBERTO j:14 semi:1.0 semf:17.0 fecha i:26062023
Docente :TAIPE CANCHO MARIO HUMBERTO j:15 semi:1.0 semf:17.0 fecha i:03072023
Docente :TAIPE CANCHO MARIO HUMBERTO j:16 semi:1.0 semf:17.0 fecha i:10072023
Docente :TAIPE CANCHO MARIO HUMBERTO j:17 semi:1.0 semf:17.0 fecha i:17072023
Docente :TAIPE CANCHO MARIO HUMBERTO j:1 semi:1.0 semf:17.0 fecha i:28032023
Docente :TAIPE CANCHO MARIO HUMBERTO j:2 semi:1.0 semf:17.0 fecha i:04042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:3 semi:1.0 semf:17.0 fecha i:11042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:4 semi:1.0 semf:17.0 fecha i:18042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:5 semi:1.0 semf:17.0 fecha i:25042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:6 semi:1.0 semf:17.0 fecha i:0

Docente :TAIPE CANCHO MARIO HUMBERTO j:15 semi:1.0 semf:17.0 fecha i:07072023
Docente :TAIPE CANCHO MARIO HUMBERTO j:16 semi:1.0 semf:17.0 fecha i:14072023
Docente :TAIPE CANCHO MARIO HUMBERTO j:17 semi:1.0 semf:17.0 fecha i:21072023
Docente :TAIPE CANCHO MARIO HUMBERTO j:1 semi:1.0 semf:17.0 fecha i:31032023
Docente :TAIPE CANCHO MARIO HUMBERTO j:2 semi:1.0 semf:17.0 fecha i:07042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:3 semi:1.0 semf:17.0 fecha i:14042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:4 semi:1.0 semf:17.0 fecha i:21042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:5 semi:1.0 semf:17.0 fecha i:28042023
Docente :TAIPE CANCHO MARIO HUMBERTO j:6 semi:1.0 semf:17.0 fecha i:05052023
Docente :TAIPE CANCHO MARIO HUMBERTO j:7 semi:1.0 semf:17.0 fecha i:12052023
Docente :TAIPE CANCHO MARIO HUMBERTO j:8 semi:1.0 semf:17.0 fecha i:19052023
Docente :TAIPE CANCHO MARIO HUMBERTO j:9 semi:1.0 semf:17.0 fecha i:26052023
Docente :TAIPE CANCHO MARIO HUMBERTO j:10 semi:1.0 semf:17.0 fecha i:0206

Docente :PEREZ CASAS EVELIN LISSET j:5 semi:1.0 semf:17.0 fecha i:29042023
Docente :PEREZ CASAS EVELIN LISSET j:6 semi:1.0 semf:17.0 fecha i:06052023
Docente :PEREZ CASAS EVELIN LISSET j:7 semi:1.0 semf:17.0 fecha i:13052023
Docente :PEREZ CASAS EVELIN LISSET j:8 semi:1.0 semf:17.0 fecha i:20052023
Docente :PEREZ CASAS EVELIN LISSET j:9 semi:1.0 semf:17.0 fecha i:27052023
Docente :PEREZ CASAS EVELIN LISSET j:10 semi:1.0 semf:17.0 fecha i:03062023
Docente :PEREZ CASAS EVELIN LISSET j:11 semi:1.0 semf:17.0 fecha i:10062023
Docente :PEREZ CASAS EVELIN LISSET j:12 semi:1.0 semf:17.0 fecha i:17062023
Docente :PEREZ CASAS EVELIN LISSET j:13 semi:1.0 semf:17.0 fecha i:24062023
Docente :PEREZ CASAS EVELIN LISSET j:14 semi:1.0 semf:17.0 fecha i:01072023
Docente :PEREZ CASAS EVELIN LISSET j:15 semi:1.0 semf:17.0 fecha i:08072023
Docente :PEREZ CASAS EVELIN LISSET j:16 semi:1.0 semf:17.0 fecha i:15072023
Docente :PEREZ CASAS EVELIN LISSET j:17 semi:1.0 semf:17.0 fecha i:22072023
Docente :PEREZ CA

Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:8 semi:1.0 semf:17.0 fecha i:17052023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:9 semi:1.0 semf:17.0 fecha i:24052023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:10 semi:1.0 semf:17.0 fecha i:31052023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:11 semi:1.0 semf:17.0 fecha i:07062023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:12 semi:1.0 semf:17.0 fecha i:14062023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:13 semi:1.0 semf:17.0 fecha i:21062023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:14 semi:1.0 semf:17.0 fecha i:28062023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:15 semi:1.0 semf:17.0 fecha i:05072023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:16 semi:1.0 semf:17.0 fecha i:12072023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:17 semi:1.0 semf:17.0 fecha i:19072023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:1 semi:1.0 semf:17.0 fecha i:30032023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:2 semi:1.0 semf:17.0 fecha i:06042023
Docente :ALVAREZ BERNAOLA LUIS ARMANDO j:3 s

Docente :SUAREZ YAURI ELADIA j:1 semi:1.0 semf:17.0 fecha i:30032023
Docente :SUAREZ YAURI ELADIA j:2 semi:1.0 semf:17.0 fecha i:06042023
Docente :SUAREZ YAURI ELADIA j:3 semi:1.0 semf:17.0 fecha i:13042023
Docente :SUAREZ YAURI ELADIA j:4 semi:1.0 semf:17.0 fecha i:20042023
Docente :SUAREZ YAURI ELADIA j:5 semi:1.0 semf:17.0 fecha i:27042023
Docente :SUAREZ YAURI ELADIA j:6 semi:1.0 semf:17.0 fecha i:04052023
Docente :SUAREZ YAURI ELADIA j:7 semi:1.0 semf:17.0 fecha i:11052023
Docente :SUAREZ YAURI ELADIA j:8 semi:1.0 semf:17.0 fecha i:18052023
Docente :SUAREZ YAURI ELADIA j:9 semi:1.0 semf:17.0 fecha i:25052023
Docente :SUAREZ YAURI ELADIA j:10 semi:1.0 semf:17.0 fecha i:01062023
Docente :SUAREZ YAURI ELADIA j:11 semi:1.0 semf:17.0 fecha i:08062023
Docente :SUAREZ YAURI ELADIA j:12 semi:1.0 semf:17.0 fecha i:15062023
Docente :SUAREZ YAURI ELADIA j:13 semi:1.0 semf:17.0 fecha i:22062023
Docente :SUAREZ YAURI ELADIA j:14 semi:1.0 semf:17.0 fecha i:29062023
Docente :SUAREZ YAURI ELADIA 

In [2]:
##SCRIPT EXTRAER TODOS LOS LINK DE CARGA LECTIVA Y NO LECTIVA
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.common.exceptions import NoSuchElementException
import time
asig = []
ciclo = []
t = []
s = []
c = []
docen = []
cl = []
cnl = []
av=[]
baseURL="https://sivireno.undc.edu.pe/"
##la cantidad que te muestra el sivireno
cantlist=100

###########################################CREACION DE METODOS#################################################################

def addLinkExcel():
    df = pd.DataFrame({ 'ASIGNATURA': asig,'CICLO': ciclo,'TURNO': t,'SECCION': s,'CANT': c,'DOCENTE': docen,'CL': cl,'CNL': cnl,'AV':av})
    df = df[['ASIGNATURA', 'CICLO','TURNO','SECCION','CANT','DOCENTE','CL','CNL','AV']]
    writer = pd.ExcelWriter('EXCEL_GENERADOS/CREATE_LIST_CURSOS_AGRONOMIA'+time.strftime("%Y%m%d-%H-%M-%S")+'.xlsx')
    df.to_excel(writer, 'Sheet', index=False)
    #writer.save()
    writer.close()
    print("TERMINO PROCESO")
###########################################CREACION DE METODOS#################################################################
try:
    for j in range(int(cantlist)):
        
        #SOLO PARA LA MODALIDAD DEL CURSO EN M
                                                    
        if driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[3]').text =="M":
            asig.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[4]').text)
            ciclo.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[5]').text)
            t.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[6]').text)
            s.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[7]').text)
            c.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[8]').text)
            docen.append(driver.find_element(by=By.XPATH, value='//*[@id="app"]/div[2]/table/tbody/tr['+str(j+1)+']/td[9]').text)
            cl.append(baseURL+driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(10) > a:nth-child(1)").getAttribute("href");'))
            cnl.append(baseURL+driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(10) > a:nth-child(2)").getAttribute("href");'))
            av.append(driver.execute_script('return document.querySelector("#app > div.table-responsive > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(13) > a").getAttribute("href");'))
                                                        
    addLinkExcel()
except:
    addLinkExcel()


TERMINO PROCESO
